<h3> Você fará um divertido jogo de campo minado. <br>
 Para isto, existe um arquivo de texto chamado “field.csv” contendo x colunas e y linhas, com “X” e “   ”. <br>
 “X” indica as bombas e “   ” indica os espaços válidos. Em uma lista de listas, será carregado o conteúdo <br>
 do arquivo CSV e o usuário será perguntado sobre qual posição deseja arriscar. <br>
 Caso a posição informada tenha uma bomba, ele perde o jogo. <br>
 Caso a posição contenha um espaço livre, ele poderá continuar.<br></h3>

In [1]:
import csv
from random import randint

In [2]:
def criar_arquivo():
    try:
        with open("Campominado/vencedores.csv") as f:
            pass
    except FileNotFoundError:
        try:
            with open("Campominado/vencedores.csv", "w+") as f:
                keys = ["Data","Campeão", "Nível"]
                file = csv.DictWriter(f, delimiter = "\t", fieldnames = keys)
                file.writeheader()
        except FileNotFoundError:
            from os import mkdir
            mkdir("Campominado")
            return criar_arquivo()

In [3]:
def save_data(dicio):
    with open("Campominado/field.csv", "w+") as f:
        headers = [key for key in dicio.keys()]
        file = csv.DictWriter(f, delimiter="\t", fieldnames=headers)
        file2 = csv.writer(f, delimiter = "\t")
        file.writeheader()
        for linha, _ in enumerate(dicio[1]):
            lin = []
            for col in dicio.keys():
                v = dicio[col]
                lin.append(v[linha])
            file2.writerow(lin)

In [4]:
def criar_campo(x, y):
    fill = {0: " ", 1: "X"}
    dicio = {" ": [chr(coluna) for coluna in range(65, y+65)]}
    for col in range(x):
        coluna = []
        for row in range(y):
            coluna.append(fill[randint(0,1)])
        dicio[col+1] = coluna
    save_data(dicio)

In [5]:
def load_data(url):
    with open(url) as f:
        file = csv.reader(f, delimiter="\t")
        head = 0
        linhas = []
        output = []
        for x in file:
            if not head:
                head = x
            else:
                linhas.append(x[1:])
        return linhas

### b) Faça uma função chamada “verificaPosicao” que receba como parâmetro o vetor do jogo e uma posição que o usuário deseja arriscar (lembre-se de solicitar ao usuário a posição). A função retornará true caso o usuário tenha acertado uma posição válida ou já tentada antes ou false caso tenha acertado uma bomba. Observe que é preciso saber as posições que o usuário já tentou antes.

In [6]:
def validaChute(vetor,chute, x_lim, y_lim):
    def _chute(chute):
        try:
            x = ord(chute[0])-65
            x = x if x >= 0 else int(chute[0])-1
            y = int(chute[1])-1
            return x, y
        except (IndexError, ValueError):
            return (x_lim+1, y_lim+1)
    lin, col = _chute(chute)
    if lin <= x_lim and col <= y_lim:
        return (True, lin, col)
    else:
        return (False, lin, col)
        

In [7]:
def verificaPosicao(vetor, lin, col):
    if vetor[lin][col]!= 'X':
        vetor[lin][col] = str(distanciar((lin, col), vetor))
        return False
    else:
        return True

In [8]:
def distanciar(ponto, matriz):
    dists = []
    distancia = []
    for m, mat in enumerate(matriz):
        array2 = []
        for n, _ in enumerate(mat):
            if matriz[m][n] == "X":
                dist = min(abs(ponto[0] - m),abs(ponto[1] - n))
                if dist != 0:
                    array2.append(dist)
        dists.append(array2)
        for x in dists:
            distancia += x
    return min(distancia)

In [9]:
def verificaChute(chute, chutes=[]):
    if chute in chutes:
        return False
    else:
        return True

### Quando o jogador acertar todos os espaços válidos (“O”) ele ganha o jogo e seu nome irá para o “hall da fama” que é um arquivo que gravará seu nome.


In [10]:
def salvaVencedor(nome, dificuldade):
    with open("Campominado/vencedores.csv", "a+") as f:
        data = registrar_data()
        file = csv.writer(f, delimiter = "\t")
        data = registrar_data()
        registro = (data, nome, dificuldade)
        file.writerow(registro)

In [11]:
def registrar_data():
    from datetime import date
    hoje = date.today()
    dia = hoje.day
    mes = hoje.month
    ano = hoje.year
    data = f"{dia}/{mes}/{ano}"
    return data

In [12]:
def dificuldade():
    nivel = {0: {"Dificuldade":"Muito Fácil","Tamanho":(3,3)}, 1: {"Dificuldade": "Fácil", "Tamanho": (5,4)},
             2: {"Dificuldade": "Médio", "Tamanho": (5,5)}, 3: {"Dificuldade": "Díficil", "Tamanho": (6,6)},
             4: {"Dificuldade": "Lenda", "Tamanho": (7,7)}, 5: {"Dificuldade": "Bomberman", "Tamanho": (8,8)},
             6: {"Dificuldade": "Personalizado", "Tamanho": (0,0)}}
    print("Selecione dificuldade de jogo")
    for x, _ in nivel.items():
        print(f'{x} - {nivel[x]["Dificuldade"]}')
    opcao = int(input(">> "))
    if opcao in nivel:
        if opcao == 6:
            dimensoes = input("Digite COL,LIN >> ").split(",")
            dimensoes = (int(x) for x in dimensoes)
            criar_campo(*dimensoes)
        else:
            criar_campo(*nivel[opcao]["Tamanho"])
        return nivel[opcao]["Dificuldade"]
    else:
        return dificuldade()

In [13]:
def exibir_campo(lista):
    saida = []
    for x, y in enumerate(lista):
        aux = []
        for w, z in enumerate(y):
            if z in "X ":
                aux.append(".")
            else:
                aux.append(z)
        saida.append(aux)

    head = [" "]+[x for x in range(1, len(lista[0])+1)]
    line = [chr(x+65) for x in range(len(lista))]
    print(*head, sep = "\t", end="\n\n")
    for index, s in enumerate(saida):
        valor = [line[index]]+s
        print(*valor, sep= "\t")    

### a) Na função principal, carregue o vetor com os dados do arquivo texto

In [14]:
def limpar_tela():
    import os
    from IPython.display import clear_output
    clear_output()
    os.system(["clear", "cls"][os.name == 'nt'])

In [15]:
def main():
    criar_arquivo()
    nivel = dificuldade()
    campo = load_data("Campominado/field.csv")
    elementos = []
    for linha in campo:
        elementos += linha
    y_lim= len(campo)
    x_lim= len(elementos)/y_lim
    vazio = [x for x in elementos if x == " "]
    vazio = len(vazio)
    jogadas_feitas = []
    bomba = False
    
    #c) Faça um loop na função principal onde o jogador poderá continuar a jogar até acertar todos os 
    #espaços válidos ou uma bomba. Lembre-se de encontrar uma forma de saber a quantidade total de espaços válidos.
    
    print("\nComo jogar: \nDigite as coordenadas no formato A1, sendo a letra a coordenada linha e o número a coordenada coluna\n")
    limpar_tela()
    while not bomba and vazio > 0:
        print()
        exibir_campo(campo)
        palpite = input("\nJogue >> ").upper()
        if verificaChute(palpite, jogadas_feitas):
            jogadas_feitas.append(palpite)
            chute = validaChute(campo, palpite, x_lim, y_lim)
            if chute[0]:
                bomba = verificaPosicao(campo, chute[1], chute[2])
                if bomba:
                    print("Você explodiu, fim de jogo!!")
                else:
                    vazio -= 1
                    motivacao = {0: "Só mais uma e você será aclamado campeão", 1: "A vitória está próxima, continue assim!!", 2: "Persistir é a chave",
                                 3: "Sua intuição é fantástica", 4: "Achei que você fosse virar carne moída agora", 5: "Bomberman, é você?",
                                 6: "Muito bem", 7: "MacGawver teria inveja de seu talento", 8:"Quem precisa de detector de bombas?",
                                 9: "Será capaz de repetir esta proeza??", 10: "Sorte ou pura habilidade?", 11:"TNT, I'm dynamite!!",
                                 12: "I'm power load!!!", 13: "Watch me explode!!"}
                    msg = randint(1,13)
                    limpar_tela()
                    print(motivacao[0] if vazio == 1 else motivacao[msg])
            else:
                limpar_tela()
                print("Jogada inválida, atente-se ao intervalo")
                
        else:
            limpar_tela()
            print("Coordenada já jogada")
            
    #d) Ao fim do loop, caso o usuário tenha acertado todas as posições válidas, solicite seu nome e envie-o
    #a uma função chamada “salvaVencedor” que salvará este nome em um arquivo CSV chamado “vencedores.csv” 
    #Caso o usuário tenha perdido, basta exibir “Que pena! Tente na próxima vez”.
    
    if bomba:
        print("Que pena! Tente na próxima vez")
    else:
        print("Parabéns, você venceu. O Hall da fama saúda sua entrada!!!")
        nome = input("Informe o nome pelo qual deseja ser lembrado >> ")
        
        salvaVencedor(nome, nivel)
        print(f"{nome}, seja bem-vindo a Valhalla!!")
        

In [ ]:
main()

Será capaz de repetir esta proeza??

 	1	2	3	4	5	6	7

A	.	.	1	.	.	.	.
B	.	.	.	.	.	.	.
C	.	.	.	.	.	.	.
D	.	.	.	.	.	.	.
E	.	.	.	.	.	.	.
F	.	.	.	.	.	.	.
G	.	.	.	.	.	.	.
